# Registering mCherry to reference acquisition

This is hard because that's the only acquisition before sequencing, which means before
we have any background fluorescence. Let's first register the 2 mCherry channels.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import iss_preprocess as issp

In [14]:
project = 'becalia_rabies_barseq'
mouse = 'BRAC8498.3e'
chamber= 'chamber_07'
mcherry_prefix = 'mCherry_1'
roi = 9


In [15]:
data_path = f'{project}/{mouse}/{chamber}'
ops = issp.io.load_ops(data_path)
ref_prefix = ops['reference_prefix']
mcherry_channels = [ops['mcherry_signal_channel'], ops['mcherry_background_channel']]
ref_channel = ops['ref_ch']

In [16]:
# Make stitched image  
stitched_ref = issp.pipeline.stitch_registered(data_path, ref_prefix, roi=roi, channels=range(4))
stitched_mcherry = issp.pipeline.stitch_registered(data_path, mcherry_prefix, roi=roi,channels=mcherry_channels)

In [ ]:
# plot a downsampled version

import matplotlib.pyplot as plt
import numpy as np
import cv2

downsample = 0.1

stitched_ref_downsampled = cv2.resize(stitched_ref, (0,0), fx=downsample, fy=downsample)
stitched_mcherry_downsampled = cv2.resize(stitched_mcherry, (0,0), fx=downsample, fy=downsample)

colors = [(1,0,0), (0,1,0), (1,0,1), (0,0,1)]
st = np.dstack([stitched_mcherry_downsampled, stitched_ref_downsampled[...,ref_channel]])
vmax = np.nanpercentile(st, 99.9, axis=(0,1))
vmin = np.nanpercentile(st, 0.01, axis=(0,1))
rgb = issp.vis.to_rgb(st, colors, vmax=vmax, vmin=vmin)

fig, axes = plt.subplots(1,1, figsize=(10,10), squeeze=False)
axes[0, 0].imshow(rgb)
axes[0, 0].axis('off')
fig.tight_layout()

In [ ]:
from iss_preprocess.vis.utils import get_stack_part

xl = [11000, 18000]
yl = [8000, 15000]
st_p = get_stack_part(stitched_mcherry, xl, yl)
st_p = np.dstack([st_p, get_stack_part(stitched_ref[...,ref_channel], xl, yl)])
vmax = np.nanpercentile(st_p, 99, axis=(0,1))
vmin = np.nanpercentile(st_p, 1, axis=(0,1))
rgb = issp.vis.to_rgb(st_p, colors, vmax=vmax, vmin=vmin)
plt.figure(figsize=(10,10))
plt.imshow(rgb, extent=[xl[0], xl[1], yl[1], yl[0]])
plt.axis('off')

In [ ]:
import pandas as pd
rabies_spot = issp.io.get_processed_path(data_path).parent / 'error_corrected_barcodes_10.pkl'
spots_df = pd.read_pickle(rabies_spot)
spots_df = spots_df.query('chamber == @chamber and roi == @roi').copy()

spots_df.shape

In [ ]:
from iss_preprocess.vis.utils import get_spot_part
st_p = get_stack_part(stitched_mcherry, xl, yl)
vmax = np.nanpercentile(st_p, 99, axis=(0,1))
vmin = np.nanpercentile(st_p, 1, axis=(0,1))
rgb = issp.vis.to_rgb(st_p, colors[:2], vmax=vmax, vmin=vmin)

sp = get_spot_part(spots_df, xl, yl)
plt.figure(figsize=(20,20))
plt.imshow(rgb, extent=[xl[0], xl[1], yl[1], yl[0]])
plt.scatter(sp['x'], sp['y'], c='cyan', s=5, alpha=0.2)
plt.axis('off')